---
## 📌 Project : 하루시작 프로젝트 다중데이터 혼잡도분석 종합  📌🔸🟦✅🆕🉐
## 📌 Description : 
    🔸 서울 교통공사 역별 일별 시간대별 승하차인원 정보
    [source link] (https://data.seoul.go.kr/dataList/OA-12921/F/1/datasetView.do#)
    
## 📌 Author : Forrest Dpark (분석 담당)
## 📌 Date : 2024.05.31 ~
## 📌 Detail : 
## 📌 Updates: 
    🟦 2024.06.02 pdg : 일별 데이터 정제 
        ✅ 기존 테스트 데이터 의 일별 데이터가 없는 부분 참고하여 일별 데이터 추출 및 데이터 칼럼 정제를 시작.
    🟦 2024.06.04 pdg : ppt 만들며 정리 
        ✅ 데이터 정제중 일부 프린트 결과 명확하게 수정함. 
        ✅ 마지막 테이블에 환승역수 관련 칼럼을 추가함. 
    🟦 2024.06.09 by pdg : 군자, 상봉 검색 오류 이슈 해결 
        ✅ 모듈 대체 할수잇는 코드 대체 
        ✅ 문제파악: station_info 를 만들때 역명 별로 역사코드를 하나씩만 지정되어서 환승역 은 저장이 안됬다. 
        따라서 군자역은 5호선 코드만 저장되고 7호선 코드는 지정되지 않아 나중에 테이블 머지할때 
        사라지게 된것!! 
        또한 상봉역 같은 경우 () 표시가 있었기에 최종테이블 검색 조건이 == 이어서 사라지게된것!!
        ✅ duplicate drop 사용하여 해결. 해결된 함수 모듈화 완료
    🟦 2024.06.10 by pdg : 08년도 부터 24년까지 데이터 추가 
        ✅ 관찰결과 엑셀데이터인 17,18,19 년도 자료는 칼럼이름이 unnamed 로 나와서 수정해야함. 
        ✅ 모든데이터 받는데 1분 30 초 걸림. 
    🟦 2024.60.12 by pdg : 데이터 년도 확장, 학습테이블 호선 데이터 확장. 
        ✅ dictionary 는 git 에 올리면 안됨 . 512 Mb 분량이어서 깃에서 에러가 터지면 다지워야함... git에 영향을 받지안토록 따로 보관하고 엑셀파일은 다 지우도록 하자.. 
        🆕 호선을 확장하는 작업도 해야함.
        🆕 모든역을 학습한 모델을 만들어도 좋지 않을까? -> 모든역에대한 예측을 하는 모델을 만들것. 


In [1]:
#### Jupyter Basic Setting ####
import pandas as pd,numpy as np
import matplotlib.pyplot as plt,seaborn as sns  # 시각화
import warnings; warnings.filterwarnings('ignore')  # 경고 무시
import sys,os 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
from Module.Functions import Service
# 기본 세팅
Service.plotSetting("seaborn-v0_8")


___## OS platform 한글 세팅완료 ## ___


In [2]:
## Data dictionary SAve 
# np.save("../Data/DataDictionary/subway_dict_test.npy",subway_dict_22_23) ## 99.8Mb
subway_dict_22_23 =np.load("../Data/DataDictionary/subway_dict_test.npy",allow_pickle=True).tolist()
subway_dict_22_23.keys()

def dataInfoPrint(data):
    Service.dataInfoProcessing(data)
    return data
for key in subway_dict_22_23.keys():
    subway_dict_22_23[key]=dataInfoPrint(subway_dict_22_23[key])
    print(f"{key.split('subway')[-1]}년도 데이터 ")
    subway_dict_22_23[key].head(3)



1. Data row/colum numbers : 98603/26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			98603/98603 [uniq/raw]
	1.[수송일자(object)]: 			181/98603 [uniq/raw]
	2.[호선(object)]: 			8/98603 [uniq/raw]
	3.[역번호(int64)]: 			279/98603 [uniq/raw]
	4.[역명(object)]: 			244/98603 [uniq/raw]
	5.[승하차구분(object)]: 			2/98603 [uniq/raw]
	 ...etc (추가로 20개의 칼럼이 있습니다 )
23_0년도 데이터 

1. Data row/colum numbers : 165376/26

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[연번(int64)]: 			165376/165376 [uniq/raw]
	1.[수송일자(object)]: 			304/165376 [uniq/raw]
	2.[호선(object)]: 			8/165376 [uniq/raw]
	3.[역번호(int64)]: 			272/165376 [uniq/raw]
	4.[역명(object)]: 			239/165376 [uniq/raw]
	5.[승하차구분(object)]: 			2/165376 [uniq/raw]
	 ...etc (추가로 20개의 칼

In [21]:

# 필수 항목 check 
# coloum check 
test = subway_dict_22_23['subway23_0']
print("columns ---👇")
print(test.columns.tolist())
# 호선, 역사번호,역명, 승하차구분
# 연번은 drop 한다. 
if '연번' in test.columns.tolist():
    print(" 1. 연번을 삭제합니다. ")
    test=test.drop('연번',axis=1)
    # print(test.columns)
# 역명 -> 역이름
if '역명' in test.columns.tolist() :
    print(' 2."역명" ->"역이름", "역번호"->"역사번호".')
    test.rename({
        '역번호':'역사번호',
        '역명':'역이름'}
        ,axis = 1
        ,inplace = True 
        )
# 역번호 -> 역사번호 
test

# 호선 데이터가 integer 인지 확인 
if str(test['호선'].dtype)=='object':
    print(' 3. 호선 데이터가 object 입니다. ')
    for idx,line in enumerate(test['호선'].unique()):
        print("   -",line)
        if idx==2:
            print(" ..")
            break
    line_int=[int(linename.split("호선")[0]) for linename in test['호선']]
    print(" 😀호선을 integer 로 만듭니다.")
    print(" 3-1. 호선 을 제거한 이름 unique : ",*np.unique(line_int),sep=", ")
    test['호선'] = line_int
    print(" ✅변경된 호선 칼럼의 data type :",test['호선'].dtype)
else : 
    print('3. 호선 데이터가 integer 입니다.')
    for idx,line in enumerate(test['호선'].unique()):
        print(" -",line)
        if idx==2:
            print(" ..")
            break
## null check -> 없다고 가정 
subway=Service.dataInfoProcessing(test,replace_Nan=True,nanFillValue=0 )
#역코드 개수 체크 -> 상관없음
# stationInfo = Service.subway_info_table(subway,save=False)
print(subway.iloc[:4,:6])
# stationInfo

columns ---👇
['연번', '수송일자', '호선', '역번호', '역명', '승하차구분', '06시이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대', '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대', '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대', '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시이후']
 1. 연번을 삭제합니다. 
 2."역명" ->"역이름", "역번호"->"역사번호".
 3. 호선 데이터가 object 입니다. 
   - 1호선
   - 2호선
   - 3호선
 ..
 😀호선을 integer 로 만듭니다.
 3-1. 호선 을 제거한 이름 unique : , 1, 2, 3, 4, 5, 6, 7, 8
 ✅변경된 호선 칼럼의 data type : int64

1. Data row/colum numbers : 98603/25

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[수송일자(object)]: 			181/98603 [uniq/raw]
	1.[호선(int64)]: 			8/98603 [uniq/raw]
	2.[역사번호(int64)]: 			279/98603 [uniq/raw]
	3.[역이름(object)]: 			244/98603 [uniq/raw]
	4.[승하차구분(object)]: 			2/98603 [uniq/raw]
	5.[06시이전(int64)]: 			1471/98603 [uniq/raw]
	 ...etc (추가로 19개의 칼럼이 있습니다 )
   


1. Data row/colum numbers : 98603/25

2. null ceck 결과 Null 없는 clean data!

3. Column  Information (중복체크)
	idx.columName |				 |Colum Info(dtype)|** 
	 ----------------------------------------------------------------
	0.[수송일자(object)]: 			181/98603 [uniq/raw]
	1.[호선(int64)]: 			8/98603 [uniq/raw]
	2.[역사번호(int64)]: 			279/98603 [uniq/raw]
	3.[역이름(object)]: 			244/98603 [uniq/raw]
	4.[승하차구분(object)]: 			2/98603 [uniq/raw]
	5.[06시이전(int64)]: 			1471/98603 [uniq/raw]
	 ...etc (추가로 19개의 칼럼이 있습니다 )


,수송일자,호선,역사번호,역이름,승하차구분,06시이전,06-07시간대,07-08시간대,08-09시간대,09-10시간대,...,15-16시간대,16-17시간대,17-18시간대,18-19시간대,19-20시간대,20-21시간대,21-22시간대,22-23시간대,23-24시간대,24시이후
0,2023-01-01,1,150,서울역,승차,215,145,231,594,923,...,2655,2509,2696,2549,2462,2177,2190,1808,734,7
1,2023-01-01,1,150,서울역,하차,154,636,595,939,1306,...,2282,2295,2526,1930,1897,1487,991,609,280,46
2,2023-01-01,1,151,시청,승차,48,73,106,194,239,...,843,895,959,985,670,630,515,330,146,0
3,2023-01-01,1,151,시청,하차,64,247,293,463,522,...,602,575,533,456,285,267,246,154,79,18
4,2023-01-01,1,152,종각,승차,407,235,158,201,188,...,1145,1402,1223,1272,911,913,906,602,232,3
